## 데이터 수집 & 전처리

In [1]:
#API 호출 및 데이터 품질 검사
import re
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, FunctionMessage
from langchain_core.runnables import Runnable
from langchain.memory import ConversationBufferMemory
from math import radians, sin, cos, sqrt, atan2

## API로 데이터 불러와서 활용 (가능하면)
- https://www.safetydata.go.kr/disaster-data/view?dataSn=3287#none 에서 파일 불러오고, 활용가능하다 판단되면 활용해보기.
- 지금은 PDF 파일만 수집 활용중

In [2]:
# .env 파일 로드
load_dotenv()

# 환경 변수에서 API 키 불러오기
api_key = os.getenv("OPENAI_API_KEY")

# PDF 파일들이 저장된 디렉토리
directory_path = "SourceCode/RAG/"

# PDF 파일 로드
all_docs = []  # 모든 문서를 저장할 리스트

# 디렉토리에서 모든 파일을 순회
for file_name in os.listdir(directory_path):
    if file_name.endswith(".pdf"):  # PDF 파일만 처리
        file_path = os.path.join(directory_path, file_name)  # 파일 경로 생성
        loader = PyPDFLoader(file_path=file_path)  # PyPDFLoader 생성
        docs = loader.load()  # 문서 로드
        all_docs.extend(docs)  # 전체 문서 리스트에 추가

# print(f"총 {len(all_docs)}개의 페이지가 로드되었습니다.")

In [3]:
def preprocess_text(text):
    '''전처리 함수
    설명 : 추가할 부분 추가하기'''
    #  처음에 숫자가 나오면 제거
    text = re.sub(r'^\s*\d+\s*', '', text)   
    
    #  "비상시 국민행동요령 알아야 안전하다"로 시작하면 제거
    if text.startswith('비상시 국민행동요령 알아야 안전하다'):
        text = text[len('비상시 국민행동요령 알아야 안전하다'):].lstrip()
    
    #  특정 패턴이 시작 부분에 있으면 제거 (패턴 1)
    pattern1 = r'^(\s*만화로 보는 비상시\s*국민행동요령\s*화생방 피해대비\s*행동요령\s*인명시설 피해시\s*행동요령\s*비상대비물자\s*준비 및 사용요령\s*비상사태시\s*행동요령\s*)'
    text = re.sub(pattern1, '', text)
    
    #  특정 패턴이 시작 부분에 있으면 제거 (패턴 2)
    pattern2 = r'^(\s*온 가족이 함께\s*안전하게\s*화생방 피해대비\s*행동요령\s*인명시설 피해시\s*행동요령\s*비상대비물자\s*준비 및 사용요령\s*비상사태시\s*행동요령\s*화생방경보 발령시\s*국민행동요령\s*핵 경보 발령시\s*국민행동요령\s*)'
    text = re.sub(pattern2, '', text)
    
    #  불필요한 줄 바꿈과 공백 정리
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text)

    #  한글,. - % / ()를 제외한 모든 특수문자 제거
    text = re.sub(r'[^a-zA-Z0-9\s%.-/()\uAC00-\uD7A3]+', '', text)

    #  문자열 양쪽 공백 제거
    text = text.strip()
    
    return text


In [4]:
# 각 문서의 page_content 전처리
for doc in all_docs:
    doc.page_content = preprocess_text(doc.page_content)

In [5]:
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(all_docs)

# 청크 확인 디버그 코드
for idx, chunk in enumerate(splits):
    print(f"Chunk {idx + 1}:")
    print("-" * 20)
    print(chunk.page_content)
    print("\n" + "=" * 40 + "\n")

Chunk 1:
--------------------
RUN HIDE TELL www.nctc.go.kr


Chunk 2:
--------------------
11 12 13 14 15 16 18 19 20 21 06 07 우편물택배 테러 화학생물 테러 해외여행 중 테러 억류납치 테러 총기난사 테러 차량돌진 테러 폭발물 테러 항공기 피랍 드론 테러 다중이용시설 테러 방사능 테러 테러는 우리나라와 상관없지 않나요 세계를 뒤흔든 테러 사건 테러경보 단계 테러의심피해상황 즉시 신고 테러 신고요령 다중이용시설 신변안전 해외여행 위험국가 방문 자제 여행경보 단계 해외여행 중 피해예방 수칙 08 테러 유형별 행동요령 06 꾸준히 증가하는 테러 사건 22 테러 피해 예방 일반 수칙 34 대테러센터 소개 주요 임무 홈페이지와 유튜브 채널 테러방지법 소개 23 24 25 26 27 28 30 34 35 36 CONTENTS 테러대비 행동요령 (www.nctc.go.kr)  0302  테러대비 행동요령 (www.nctc.go.kr)


Chunk 3:
--------------------
112      04  테러대비 행동요령 (www.nctc.go.kr)


Chunk 4:
--------------------
2021 2007     2001    2015          2004 테러는 우리나라와 상관없지 않나요 최근 국내에서 테러단체에 동조하는 혐의로 적발되는 사례가 꾸준히 증가하고 있으며 해외에서는 우리 국민 대상으로 혐오 테러가 지속적으로 발생하고 있습니다. 드론을 활용하거나 차량 돌진 우편물이나 택배를 이용하는 등 새로운 유형의 테러도 우리 주변에서 언제든 발생할 수 있습니다. 테러는 국적도 인종도 지역도 가리지 않습니다. 테러에 안전지대는 없습니다. 세계를 뒤흔든 테러 사건 테러대비 행동요령 (www.nctc.go.kr)  0706  테러대비 행동요령 (www.nctc.go.kr)


Chunk 5:
--------------------
/5 /5 /5   

## 임베딩

In [6]:
# 1. OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. 로컬 파일 저장소 설정 (사용자 환경에 맞는 경로로 설정)
store = LocalFileStore("C:Alex/text-embedding")  # 로컬 경로 설정 각자 작성 해야합닏.

# 3. 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace=embeddings.model,  # 모델 이름을 네임스페이스로 설정
)


'''
# 4. 예시 텍스트
example_texts = [
    "OpenAI의 임베딩 모델은 강력한 자연어 처리를 제공합니다.",
    "로컬 캐시는 효율적인 데이터 처리를 가능하게 합니다.",
    "임베딩 모델을 활용하여 고품질의 애플리케이션을 개발할 수 있습니다."
]

# 5. 임베딩 생성 및 저장된 결과 출력
for text in example_texts:
    embedding = cached_embedder.embed_query(text)  # 임베딩 생성
    print(f"텍스트: {text}")
    print(f"임베딩 (첫 10개 값): {embedding[:10]}")  # 임베딩 값의 일부를 출력

print("임베딩 생성 완료 및 로컬 캐시에 저장되었습니다.")
'''

'\n# 4. 예시 텍스트\nexample_texts = [\n    "OpenAI의 임베딩 모델은 강력한 자연어 처리를 제공합니다.",\n    "로컬 캐시는 효율적인 데이터 처리를 가능하게 합니다.",\n    "임베딩 모델을 활용하여 고품질의 애플리케이션을 개발할 수 있습니다."\n]\n\n# 5. 임베딩 생성 및 저장된 결과 출력\nfor text in example_texts:\n    embedding = cached_embedder.embed_query(text)  # 임베딩 생성\n    print(f"텍스트: {text}")\n    print(f"임베딩 (첫 10개 값): {embedding[:10]}")  # 임베딩 값의 일부를 출력\n\nprint("임베딩 생성 완료 및 로컬 캐시에 저장되었습니다.")\n'

In [7]:
vectorstore = FAISS.from_documents(documents=splits, embedding=cached_embedder)

In [8]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4}) 

# 테스트 하면서 수정할 예정

## Fuctiong calling 함수 정의

In [9]:
# 대피소 정보 불러오기
with open("SourceCode/shelters.json", "r", encoding="utf-8") as f:
    shelters = json.load(f)

In [10]:
# shelters 리스트를 데이터프레임으로 변환
df = pd.DataFrame(shelters)
df = df[df['SHNT_PSBLTY_NOPE'] != '0']  # 대피가능인원수 0인 곳을 제외

In [11]:
print(df.head())

  GRND_UDGD_SE ORTM_UTLZ_TYPE   SGG_CD     FCLT_CD    FCLT_SE_CD  \
0            1            주차장  3910000  S202400002  3              
1            1            주차장  3910000  S202400001  3              
2            1            주차장  4020000  S202400001  3              
3            1          지하주차장  3650000  S201700010  3              
4            1          지하주차장  3650000  S201700011  3              

     ROAD_NM_CD  LOT_MIN SHNT_PSBLTY_NOPE  FCLT_SCL  LOT_SEC  ... LAT_PROVIN  \
0  3352297             6            34761     28678       24  ...         37   
1  3352295             6            37152     30651       27  ...         37   
2  3182030            56            39997     32998       42  ...         37   
3  3165028            23             6577      5426       55  ...         36   
4  3165028            23             5126      4229       56  ...         36   

  LAT_MIN MNG_INST_TELNO                     FCLT_NM  FCLT_DSGN_DAY  \
0       1  031-8024-4900  평택지제역동문굿모닝힐맘시

In [12]:
# DMS를 소수점 좌표로 변환하는 함수 (경도 위도 각각 숫자 합치기)
def dms_to_dd(degrees, minutes, seconds):
    return degrees + minutes / 60 + seconds / 3600

# 위도 변환 (LAT_PROVIN, LAT_MIN, LAT_SEC를 사용)
df['위도'] = df.apply(lambda row: dms_to_dd(
    float(row['LAT_PROVIN']), 
    float(row['LAT_MIN']), 
    float(row['LAT_SEC'])
), axis=1)

# 경도 변환 (LOT_PROVIN, LOT_MIN, LOT_SEC를 사용)
df['경도'] = df.apply(lambda row: dms_to_dd(
    float(row['LOT_PROVIN']), 
    float(row['LOT_MIN']), 
    float(row['LOT_SEC'])
), axis=1)

# 주소 컬럼 설정 (도로명 주소 사용)
df['주소'] = df['FCLT_ADDR_RONA']
# 장소 컬럼 설정
df['시설명'] = df['FCLT_NM']

# 필요한 컬럼만 선택 추가가 가능하다. (대피소 정보 불러오고 추가 정보가 필요하다고 판단되면 추가하면 됩니다.) 답변 테스트 할때, 필요하다고 판단되면
df_result = df[['시설명','주소', '위도', '경도']]

print(df_result.head())

                          시설명                                        주소  \
0  평택지제역동문굿모닝힐맘시티4단지 지하주차장 1층  경기도 평택시 신촌5로 56 (칠원동, 평택지제역동문굿모닝힐맘시티4단지)   
1  평택지제역동문굿모닝힐맘시티2단지 지하주차장 1층  경기도 평택시 신촌3로 12 (칠원동, 평택지제역동문굿모닝힐맘시티2단지)   
2             힐스테이트 금정역 지하주차장         경기도 군포시 엘에스로 143 (금정동, 힐스테이트 금정역)   
3                유등마을아파트 104동           대전광역시 중구 수침로 138 (태평동, 유등마을아파트)   
4                유등마을아파트 106동           대전광역시 중구 수침로 138 (태평동, 유등마을아파트)   

          위도          경도  
0  37.025278  127.106667  
1  37.026389  127.107500  
2  37.373333  126.945000  
3  36.332222  127.398611  
4  36.331667  127.398889  


In [13]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    위도와 경도를 받아 두 지점 사이의 거리를 킬로미터 단위로 계산하는 함수
    """
    # 위도와 경도를 라디안으로 변환
    lat1_rad, lon1_rad = radians(lat1), radians(lon1)
    lat2_rad, lon2_rad = radians(lat2), radians(lon2)

    # 위도와 경도의 차이 계산
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine 공식 적용
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    R = 6371  # 지구 반지름 (킬로미터 단위)
    distance = R * c
    return distance

## 카카오맵 API 호출

In [14]:
''' 키워드 검색시 주소를 호출하지 못함 -> 키워드 검색 로직 추가 '''

load_dotenv()
kakaoapikey = os.getenv("REST_API_KEY") # 카카오 API 호출
def get_coordinates(query):
    """
    검색 질의어(query)를 기반으로 주소 검색 API와 장소 검색 API를 순차적으로 호출하여 경도와 위도를 반환.

    Args:
        query (str): 검색할 주소 또는 장소.
        kakaoapikey (str): 카카오맵 REST API 키.

    Returns:
        tuple: (longitude, latitude) - 경도(x), 위도(y)
    """
    # 1. 주소 검색 API 호출
    address_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {kakaoapikey}"}
    params = {"query": query, "size": 1}

    address_response = requests.get(address_url, headers=headers, params=params)
    if address_response.status_code == 200:
        data = address_response.json()
        documents = data.get("documents", [])
        if documents:
            x = documents[0].get("x")
            y = documents[0].get("y")
            return (x, y)
    
    # 2. 장소 검색 API 호출 (주소 검색 결과가 없을 경우)
    keyword_url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    keyword_response = requests.get(keyword_url, headers=headers, params=params)
    if keyword_response.status_code == 200:
        data = keyword_response.json()
        documents = data.get("documents", [])
        if documents:
            x = documents[0].get("x")
            y = documents[0].get("y")
            return (x, y)
    
    # 결과 없음
    print("검색 결과가 없습니다.")
    return None


# 테스트 실행
if __name__ == "__main__":
    query = "동대구역"  # 검색할 장소 또는 주소
    coordinates = get_coordinates(query)
    if coordinates:
        print(f"경도: {coordinates[0]}, 위도: {coordinates[1]}")

경도: 128.628393775388, 위도: 35.8793239931795


In [15]:
def find_nearest_shelters(address: str) -> str:
    """
    주어진 주소를 기반으로 가까운 대피소 n개 정보를 반환하는 함수.
    """
    coordinates = get_coordinates(address)
    if coordinates:
        user_lon = float(coordinates[0])
        user_lat = float(coordinates[1])

        
        def calculate_distance(row):
            try:
                shelter_lat = float(row['위도'])
                shelter_lon = float(row['경도'])
                return haversine_distance(user_lat, user_lon, shelter_lat, shelter_lon)
            except (ValueError, TypeError):
                return None

        # 거리 계산 및 데이터프레임 업데이트
        df_result['거리'] = df_result.apply(calculate_distance, axis=1)
        df_valid = df_result[df_result['거리'].notnull()]
        df_sorted = df_valid.sort_values(by='거리')
        df_top3 = df_sorted.head(3)

        # 결과 문자열 생성
        result = "\n가장 가까운 대피소 정보 (거리순):"
        # 추후에 대피소 정보를 가져오는 테스트할때, 코드 변경할 수 있습니다.
        for idx, row in df_top3.iterrows():
            result += f"\n\n[{idx+1}]"
            result += f"\n시설명: {row['시설명']}"
            result += f"\n주소: {row['주소']}"
            result += f"\n현재 위치로부터의 거리: {row['거리']:.2f} km"

            shelter_add = row['주소'].replace(' ', '') 
            kakao_map_link = f"https://map.kakao.com/link/search/{shelter_add}"
            result += f"\n지도 링크: {kakao_map_link}"
        return result
    else:
        return "좌표를 가져올 수 없습니다."


In [16]:
# 함수 스키마 정의
functions = [
    {
        "name": "find_nearest_shelters",
        "description": "주어진 주소를 기반으로 가장 가까운 대피소 정보를 반환하는 함수.",
        "parameters": {
            "type": "object",
            "properties": {
                "address": {
                    "type": "string",
                    "description": "대피소를 찾고자 하는 주소"
                }
            },
            "required": ["address"]
        }
    }
]

## Streamlit 에서 위치기반 서비스 API를 이용하여, 사용자 위치를 불러와서 대피소 정보를 반환하는 것으로 추가기능 구현 예정임

In [17]:
# 모델 정의
api_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    model_kwargs={"functions": functions}
)

In [18]:
#프롬프트 자료들 추후 삭제
'''
# 1
chat_template = ChatPromptTemplate.from_messages(
    [
        # system 메시지로 챗봇 역할 정의
        ("system", "당신은 재난안전 전문 AI 어시스턴트입니다. "
                   "사용자가 요청한 재난 상황에 대해 적절한 정보를 제공하고, 추가로 도움이 될 만한 리소스를 추천합니다."),
        
        # 인간 사용자 메시지 및 예시 응답
        ("human", "안녕하세요!"),
        ("ai", "안녕하세요! 저는 재난 상황에서 필요한 정보를 제공하기 위해 존재하는 AI입니다. 무엇을 도와드릴까요?"),
        
        # 사용자 질문 및 AI의 다이내믹한 응답
        ("human", "{user_input}"),
    ]
)
# 2
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 사용자의 위치를 기반으로 대피소 정보를 제공하는 AI입니다. "
                   "대피소 이름, 주소, 연락처, 추가 유의사항 등을 안내합니다."),
        ("human", "안녕하세요!"),
        ("ai", "안녕하세요! 저는 가까운 대피소를 찾는 데 도움을 줄 수 있는 AI입니다. 위치 정보를 알려주시면 안내드리겠습니다."),
        ("human", "{user_input}"),
    ]
)

# 3 
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 재난 대비에 대한 정보를 전문적으로 안내하는 AI입니다. "
                   "사용자의 요청에 따라 적절한 예방 조치, 행동 요령, 추가 자료를 제공합니다."),
        ("human", "안녕하세요!"),
        ("ai", "안녕하세요! 저는 재난 대비에 필요한 정보를 제공하기 위해 여기 있습니다. 어떤 정보를 찾고 계신가요?"),
        ("human", "{user_input}"),
    ]
)
system_prompt = ("당신은 비상사태 대처 매뉴얼 전문 챗봇입니다. 재난 상황(지진, 화재, 홍수, 전쟁 등)이 발생했을 때 사용자가 안전하게 대피할 수 있도록 최적의 정보를 제공하는 것이 목표입니다.
아래의 지침에 따라 응답하세요:
1. 역할 정의: 사용자에게 비상사태에 대한 신뢰할 수 있는 정보를 제공하고, 필요한 경우 함수 호출을 통해 가장 가까운 대피소를 추천하세요.
2. 대화 스타일:
   - 간결하고 명확하며 사용자 친화적인 언어를 사용하세요.
   - 긴급 상황에 맞는 전문적인 톤을 유지하되, 사용자를 진정시키는 어조를 포함하세요.
3. 정보 제공:
   - 대피소 이름, 주소, 연락처, 운영 시간, 수용 가능 인원을 포함하여 상세 정보를 제공하세요.
   - 사용자의 현재 위치를 기반으로 최적의 대피소를 추천하세요.
   - 각 상황(지진, 화재, 홍수 등)에 맞는 행동 요령과 필요한 준비물(응급 키트 등)을 안내하세요.
4. 긴급 연락처:
   - 사용자가 추가적인 도움이 필요할 경우, 즉시 긴급 연락처(예: 119, 112)를 안내하세요.
5. 정보의 정확성과 최신성:
   - 제공된 컨텍스트와 최신 데이터를 결합하여 응답하세요.
   - 데이터가 부족한 경우, 사용자를 가장 안전한 방향으로 안내하고 추가적인 도움을 요청하도록 권장하세요.
6. 함수 호출 지침:
   - 사용자의 질문이 대피소 검색이나 위치 관련 정보를 요구하는 경우, 적절한 함수 호출을 통해 데이터를 검색하세요.
   - 함수 호출 결과를 기반으로 사용자의 요청에 대해 정확하고 상세한 응답을 생성하세요.
7. 다양한 사용자 고려:
   - 대화의 접근성을 높이기 위해 복잡한 용어 대신 쉬운 표현을 사용하세요.
8. 추가 지침:
   - 필요 시 사용자의 질문을 명확히 하기 위해 간단한 질문을 되묻는 방식을 사용하세요.
   - 제공한 정보가 명확하고 이해하기 쉬운지 항상 점검하세요."
)

'''

'\n# 1\nchat_template = ChatPromptTemplate.from_messages(\n    [\n        # system 메시지로 챗봇 역할 정의\n        ("system", "당신은 재난안전 전문 AI 어시스턴트입니다. "\n                   "사용자가 요청한 재난 상황에 대해 적절한 정보를 제공하고, 추가로 도움이 될 만한 리소스를 추천합니다."),\n        \n        # 인간 사용자 메시지 및 예시 응답\n        ("human", "안녕하세요!"),\n        ("ai", "안녕하세요! 저는 재난 상황에서 필요한 정보를 제공하기 위해 존재하는 AI입니다. 무엇을 도와드릴까요?"),\n        \n        # 사용자 질문 및 AI의 다이내믹한 응답\n        ("human", "{user_input}"),\n    ]\n)\n# 2\nchat_template = ChatPromptTemplate.from_messages(\n    [\n        ("system", "당신은 사용자의 위치를 기반으로 대피소 정보를 제공하는 AI입니다. "\n                   "대피소 이름, 주소, 연락처, 추가 유의사항 등을 안내합니다."),\n        ("human", "안녕하세요!"),\n        ("ai", "안녕하세요! 저는 가까운 대피소를 찾는 데 도움을 줄 수 있는 AI입니다. 위치 정보를 알려주시면 안내드리겠습니다."),\n        ("human", "{user_input}"),\n    ]\n)\n\n# 3 \nchat_template = ChatPromptTemplate.from_messages(\n    [\n        ("system", "당신은 재난 대비에 대한 정보를 전문적으로 안내하는 AI입니다. "\n                   "사용자의 요청에 따라 적절한 예방 조치, 행동 요령, 추가 자료를 제공합니다."),

In [ ]:
# 비상사태 대처 매뉴얼 전문 챗봇 프롬프트 템플릿 정의
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", (
            "당신은 비상사태 대처 매뉴얼 전문 챗봇입니다. "
            "재난 상황(지진, 화재, 홍수, 전쟁 등)이 발생했을 때 사용자가 안전하게 대피할 수 있도록 최적의 정보를 제공하는 것이 목표입니다.\n\n"
            "제공된 컨텍스트만 사용해서, 질문에 답변하세요."
            "아래의 지침에 따라 응답하세요:\n"
            "1. 역할 정의: 사용자에게 신뢰할 수 있는 정보를 제공하고, 필요한 경우 함수 호출을 통해 가장 가까운 대피소를 추천하세요.\n"
            "2. 대화 스타일: 간결하고 명확하며 사용자 친화적인 언어를 사용하고, 긴급 상황에 맞는 전문적인 톤을 유지하세요.\n"
            "3. 긴급 연락처: 추가적인 도움이 필요할 경우 즉시 긴급 연락처(예: 119, 112)를 안내하세요.\n"
            "4. 정보의 정확성과 최신성: 최신 데이터를 결합해 응답하세요. 데이터 부족 시 안전한 방향으로 안내하고 추가 도움을 요청하도록 권장하세요.\n"
            "5. 함수 호출 지침: 대피소 검색이나 위치 관련 질문에 적절한 함수를 호출하여 데이터를 검색하세요.\n"
            "6. 다양한 사용자 고려: 복잡한 용어 대신 쉬운 표현을 사용하세요.\n"
            "7. 추가 지침: 필요한 경우 질문을 되묻고, 제공 정보가 명확한지 점검하세요."
        )),
        ("ai", "안녕하세요! 저는 비상사태에서 안전한 대처를 도와드리는 전문 AI 챗봇입니다. 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

def get_user_input():
    return input("사용자: ")

def handle_function_call(response):
    function_call = response.additional_kwargs.get("function_call")
    if function_call:
        function_name = function_call["name"]
        function_args = json.loads(function_call["arguments"])

        if function_name == "find_nearest_shelters":
            function_result = find_nearest_shelters(**function_args)
            if function_result == "좌표를 가져올 수 없습니다.": print(function_result)
            else: print(f"챗봇: 가장 가까운 대피소는 {function_result} \n입니다.")
        else:
            print(f"알 수 없는 함수 호출: {function_name}")
    else:
        print(f"챗봇: {response.content}")


def process_response(response):
    if response.additional_kwargs.get("function_call"):
        handle_function_call(response)
    else:
        print(f"챗봇: {response.content}")

class ModelInvocation(Runnable):
    def invoke(self, input, config=None):
        messages = chat_template.format_messages(user_input=input)
        return model.invoke(messages)

class ResponseProcessor(Runnable):
    def invoke(self, input, config=None):
        process_response(input)
        return input

chat_chain = (
    ModelInvocation()
    | ResponseProcessor()
)

def main():
    # 프로그램 시작 시 초기 AI 메시지를 출력합니다.
    initial_messages = chat_template.format_messages(user_input="")
    response = model.invoke(initial_messages)
    process_response(response)

    while True:
        user_input = get_user_input()
        if user_input.lower() in ["종료", "exit", "quit"]:
            print("대화를 종료합니다.")
            break
        response = chat_chain.invoke(user_input)

if __name__ == "__main__":
    main()

챗봇: 비상 상황에서 도움이 필요하시면 언제든지 말씀해 주세요. 대피소를 찾거나 안전 지침이 필요하시면 알려주세요.
챗봇: 비상 상황에서 도움이 필요하시면 언제든지 말씀해 주세요. 대피소를 찾거나 안전 지침이 필요하시면 알려주세요.


### RAG 데이터 잘 불러오는지 확인하고, 이전 대화 메모리 활용방안